In [1]:
import pdfplumber
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import json
import os
import openai
from openai import OpenAI

api_key = "sk"
client = OpenAI(api_key = api_key)

/Users/chengruzou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/chengruzou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
import pdfplumber
import pandas as pd

client = OpenAI(api_key = api_key)

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"

extract_text_from_pdf(pdf_path)

'BBA PROGRAM\nCATALOG\n2023-2024\n1300 Clifton Road, Atlanta, GA 30322\nTel: 404-727-8106\nTABLE OF CONTENTS\nBBA Mission Statement 3\nBBA Program Staff 4\nMatriculation 6\n• Matriculation Process 6\n• Prerequisite Courses 8\n• General Education Requirements 8\nThe Goizueta BBA Program Curriculum 11\n• BBA Requirements 11\n• Area Depths – Class of 2024 13\n• Area Depths – Class of 2025 and Beyond 24\n• Concentrations 33\n• BBA Joint and Dual Degree Programs 35\n• Second Majors and Minors in Emory College 37\n• Business Minor 37\nCourse Descriptions 38\nAcademic Standards 59\nAcademic Policies and Procedures 60\nHonors and Awards 68\nBBA Life and Leadership 72\nBBA Career Management Center 79\nGoizueta Business Library 79\nFinancial Information 80\nAcademic Calendar 82\nDepartment of Veterans Affairs Pending Payment Rights 83\nEmory University is an equal opportunity/equal access/affirmative action employer fully committed to achieving a diverse\nworkforce and complies with all applicab

In [16]:
def process_text_with_llm(text):
    prompt = f"""
    Extract all relationships from the following text and format them as a JSON array of objects.
    Each object should contain 'concentration'(eg. MARKETING,ACCOUNTING..., with in the pdf there should be description of what the concentration is eg."FINANCE
Our goal in the Finance area is to provide you with a strong framework for financial decision making..." The description should be short summary)
    'program'(eg. ACCOUNTING AREA DEPTH,),
    'Required Course'(Should include all 'course_code' that are required in the depth.'couse_code'(eg. ACT 300)),
    'relationship' fields.
    'relationship' field describes the type of relationship (e.g., 'requires'.., 'includes'..., 'part of'..., for ).
    The chat should be able to extract all important relational information
    The format should be able to be printed
    Here is the text:
    {text}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content":prompt}
        ],
        
        max_tokens=4095,
        temperature=0.5
    )
    return response.choices[0].message.content


# Example usage
pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
llm_response = process_text_with_llm(pdf_text)

# Print the JSON structure
print(llm_response)

```json
[
  {
    "concentration": "ACCOUNTING",
    "description": "Accounting is the language of business, and fluency in financial reporting, the legal environment, and taxation should be part of your vocabulary.",
    "program": "ACCOUNTING AREA DEPTH",
    "Required Course": [
      "ACT 300",
      "ACT 312A",
      "ACT 312B",
      "ACT 330",
      "ACT 399R",
      "ACT 420",
      "ACT 516",
      "ACT 650",
      "ACT 613",
      "ACT 615",
      "ACT 616",
      "ACT 618"
    ],
    "relationship": "requires"
  },
  {
    "concentration": "FINANCE",
    "description": "Our goal in the Finance area is to provide you with a strong framework for financial decision making, from the Nobel Prize-winning models of the early theorists to the state-of-the-art methodologies of today.",
    "program": "FINANCE AREA DEPTH",
    "Required Course": [
      "FIN 323"
    ],
    "relationship": "requires"
  },
  {
    "concentration": "INFORMATION SYSTEMS AND OPERATIONS MANAGEMENT",
    "d

In [17]:
import json
import openai  # Make sure to have the OpenAI library installed and configured

def process_text_with_llm(text):
    prompt = f"""
    Extract all course relationships from the following text and format them as a JSON array of objects.
    Each object should contain 'course_id', 'prerequisites', and 'concentration_track' fields.
    The 'course_id' field should be the course code (e.g., ACT 300).
    The 'prerequisites' field should list all prerequisite courses (e.g., ['ACT 101', 'ACT 201']).
    The 'concentration_track' field should indicate the concentration track to which the course belongs (e.g., 'Finance').
    Example output format:
    [
      {{
        "course_id": "ACT 300",
        "prerequisites": ["ACT 101", "ACT 201"],
        "concentration_track": "Accounting"
      }},
      {{
        "course_id": "FIN 301",
        "prerequisites": ["FIN 101"],
        "concentration_track": "Finance"
      }},
      ...
    ]
    The format should be able to be printed.
    Here is the text:
    {text}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content":prompt}
        ],
        
        max_tokens=4095,
        temperature=0.5
    )
    return response.choices[0].message.content


# Example usage
pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
llm_response = process_text_with_llm(pdf_text)

# Print the JSON structure
print(llm_response)


```json
[
  {
    "course_id": "ACT 300",
    "prerequisites": ["ACT 200"],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 312A",
    "prerequisites": ["ACT 200"],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 312B",
    "prerequisites": ["ACT 312A"],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 330",
    "prerequisites": [],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 399R",
    "prerequisites": [],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 410",
    "prerequisites": [],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 420",
    "prerequisites": [],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 499R",
    "prerequisites": [],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 516",
    "prerequisites": ["ACT 200"],
    "concentration_track": "Accounting"
  },
  {
    "course_id": "ACT 650",

In [37]:
import json
import openai  # Make sure to have the OpenAI library installed and configured

def process_text_with_llm(text):
    prompt = f"""
Extract all relationships from the following text and format them as a JSON array of objects.
Each object should contain the following fields:
- 'concentration': The concentration name (e.g., MARKETING, ACCOUNTING, ISOM, MARKETING, ORGANIZATION AND MANGEMENT, INTERDISCIPLINARY AREA ).
- 'description': A short summary of the concentration (e.g., "Our goal in the Finance area is to provide you with a strong framework for financial decision making...").
- 'program': The program name (e.g., ACCOUNTING AREA DEPTH).
- 'required_courses': An array of required course codes (e.g., ['ACT 300', 'FIN 320']).
- 'electives': An array of elective course details including 'course_code' and 'course_name' (e.g., [ "'course_code': 'ACT 300', 'course_name': 'Business Analysis and Valuation'"]).
- 'subsitute elective' : course the the title "May substitute ONE of the following related courses for an elective" for each concentration. 
- 'number of elective required": the number that is required for the specific concentration

Format the output as a JSON array of objects. Here is the text:

{text}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content":prompt}
        ],
        
        max_tokens=4096,
        temperature=0.5
    )
    return response.choices[0].message.content


# Example usage
pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
llm_response = process_text_with_llm(pdf_text)

# Print the JSON structure
print(llm_response)


```json
[
  {
    "concentration": "ACCOUNTING",
    "description": "Accounting is the language of business, and fluency in financial reporting, the legal environment, and taxation should be part of your vocabulary. Every business decision involves choices of transaction structure, resource allocation, reporting to constituents such as shareholders, creditors, and investors, and compliance with many levels of government regulation and taxation. The Accounting area depth gives you critical skills to help you make complex decisions in an ever-changing environment.",
    "program": "ACCOUNTING AREA DEPTH",
    "required_courses": [],
    "electives": [
      {"course_code": "ACT 300", "course_name": "Business Analysis and Valuation"},
      {"course_code": "ACT 312A", "course_name": "Financial Reporting of Operations M,P (formerly Financial Reporting I)"},
      {"course_code": "ACT 312B", "course_name": "Financial Reporting of Debt & Equity M,P (formerly Financial Reporting II)"},
      

In [42]:
import json
import openai  

def process_text_with_llm(text):
    prompt = f"""
Extract all relationships from the following text and format them as a JSON array of objects.
Each object should contain the following fields:
- 'Prerequisite Course for pre-business student': need to specify economic requirment, statistical competency... (e.g., Statistical Compentency: ['AP Statistics', 'One course from(EOCN 220, QTM 100, QTM 110)', online bootcamp])
- 'General Education Requirement (Gold: Before class of 2027 )': specify each Area (from 1 to 10), print out number og the course requirement, and specify a example course: (e.g. FWR: one course [Example ccourse: ENG 101] )
- 'General Education Requirement (Blue: After class of 2027)' : specific which class should be take in which year in the university (e.g., First year: ECS 101, HLTH 100...)
- 'BBA Requirement (Class of 2024)': Read the BBA Core, professional development core, and elective distribution. print the elective distribution like the chart
- 'BBA requirement (Class of 2025 and Beyond)": Read the BBA Core, Flex core, Professional Development and Co-curricular core, Elective(brief summary), immersive experient elective (brief summary).

Format the output as a JSON array of objects. Here is the text:

{text}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content":prompt}
        ],
        
        max_tokens=4096,
        temperature=0.5
    )
    return response.choices[0].message.content


# Example usage
pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
llm_response = process_text_with_llm(pdf_text)

# Print the JSON structure
print(llm_response)

```json
[
  {
    "Prerequisite Course for pre-business student": {
      "Economic Requirement": [
        "FIN 201 Business Economics (for Emory College students)",
        "ECON 101 Microeconomics OR AP Micro credit AND ECON 112 Macroeconomics OR AP Macro credit",
        "IB credit for Economics (Higher Level)"
      ],
      "Statistical Competency": [
        "AP Statistics",
        "One course from (ECON 220, QTM 100, QTM 110, MATH_OX 117, or an equivalent course by petition)",
        "Successful passage of a statistics exam following completion of an online bootcamp"
      ],
      "Additional Requirements": [
        "BUS 350 Data and Decision Analytics",
        "BUS 290 Tech Tools A: Excel (1 credit)"
      ]
    }
  },
  {
    "General Education Requirement (Gold: Before class of 2027)": [
      {
        "Area": "I",
        "Requirement": "First Year Seminar (FSM)",
        "Example Course": "First Year Seminar taken at Emory College"
      },
      {
        "Area": "I

In [41]:
def segment_text(text, max_length=15000):
    # Split text into segments with a maximum length
    segments = []
    current_segment = ""
    for paragraph in text.split('\n'):
        if len(current_segment) + len(paragraph) + 1 > max_length:
            segments.append(current_segment)
            current_segment = paragraph + "\n"
        else:
            current_segment += paragraph + "\n"
    if current_segment:
        segments.append(current_segment)
    return segments
    
def process_text_with_llm(text):
    prompt = f"""
Extract all relationships from the following text and format them as a JSON array of objects.
Each object should contain the following fields:
- 'concentration': The concentration name (e.g., MARKETING, ACCOUNTING, ISOM, MARKETING, ORGANIZATION AND MANGEMENT, INTERDISCIPLINARY AREA ).
- 'description': A short summary of the concentration (e.g., "Our goal in the Finance area is to provide you with a strong framework for financial decision making...").
- 'program': The program name (e.g., ACCOUNTING AREA DEPTH).
- 'required_courses': An array of required course codes (e.g., ['ACT 300', 'FIN 320']).
- 'electives': An array of elective course details including 'course_code' and 'course_name' (e.g., [ "'course_code': 'ACT 300', 'course_name': 'Business Analysis and Valuation'"]).
- 'subsitute elective' : course the the title "May substitute ONE of the following related courses for an elective" for each concentration. 
- 'number of elective required": the number that is required for the specific concentration

Format the output as a JSON array of objects. Here is the text:

{text}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content":prompt}
        ],
        
        max_tokens=4096,
        temperature=0.2
    )
    return response.choices[0].message.content


pdf_path = "/Users/chengruzou/Desktop/AI.Xexperience/student-handbook.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
text_segments = segment_text(pdf_text)

# Process each text segment and combine the results
combined_results = []
for segment in text_segments:
    result = process_text_with_llm(segment)

# Print the combined results in a readable format
print(json.dumps(combined_results, indent=4))

[]
